In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import os, re, gc
import h5py
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import roc_curve, precision_recall_curve, roc_auc_score

batch = 256; n_pols = 9
path_dir = '/mnt/ML-drive/Artem/Carbon_test/'
class_names_ft = [k for k in os.listdir(path_dir) if ('C60' in k or 'C100' in k)]
test_types = ['sig','bckg']
id_header = ['HeaderID','ViewID','GrainID','pol0','pol1','pol2','pol3','pol4','pol5','pol6','pol7','tr_flag','n_pol']
feat_names = ['x','y','z','lx','ly','phi','npx','vol','eps']

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
from keras.models import load_model
import keras.backend as K
def swish(x):
    """
    x*sigmoid(x)
    """
    return (K.sigmoid(x) * x)

def short_name(name):
    if 'keV' in name: return 'Carbon/'+(name.split('_')[0])[1:]
    return name

Using TensorFlow backend.


In [2]:
%%time
model = {}
dir_path = {'c100':'C100keV_WithFilter_Blue_/','c60':'C60keV_H_2018Oct04_WithFilter_Blue_/','c30':'C30keV_H_withFilter_Blue_Exposed1500_Re_/'}
model_type = {'c100':'no_rot/conv4_3d_res_Carbon-100keV_fog','c60':'no_rot/conv4_3d_res_Carbon-60keV_fog','c30':'no_rot/conv4_3d_res_Carbon-30keV_fog'}
for carb in ['c100','c60','c30']:
    model[carb] = load_model('/mnt/ML-drive/Artem/Python/NEWS/CNN/networks/res_v3/e50/'+model_type[carb]+'.h5', custom_objects={'swish':swish})

CPU times: user 36.8 s, sys: 748 ms, total: 37.5 s
Wall time: 44.2 s


In [3]:
preds = {}
start = datetime.now()
for name in class_names_ft:
    name_ = (name.split('k')[0]).lower()
    for tt in test_types:
        fold = datetime.now()
        with h5py.File('data_9pol.h5', 'r') as dfile:
            x_test = dfile[short_name(name)+'/'+tt+'/images'][...]
            preds[name_+'-'+tt] = model[name_].predict(x_test[...,np.newaxis], batch_size=batch, verbose=1)
            preds[name_+'-'+tt] = np.hstack((dfile[short_name(name)+'/'+tt+'/pol_ids'],preds[name_+'-'+tt]))
        with open(dir_path[name_]+name_+'_'+tt+'_pred_yandex.txt','w') as pfile:
            for line in preds[name_+'-'+tt]:
                line = ','.join([str(k) if (k-int(k)) else str(int(k)) for k in line])
                pfile.write(line+'\n')
        print(short_name(name)+'-'+tt,' predict time:',datetime.now()-fold,'\n')
print('total predict time:',datetime.now()-start)

5982/5982 [==============================] - 8s 1ms/step
Carbon/100keV-sig  predict time: 0:00:08.787481 

5850/5850 [==============================] - 3s 441us/step
Carbon/100keV-bckg  predict time: 0:00:03.037835 

3911/3911 [==============================] - 3s 774us/step
Carbon/60keV-sig  predict time: 0:00:03.332687 

5627/5627 [==============================] - 3s 445us/step
Carbon/60keV-bckg  predict time: 0:00:02.986634 

total predict time: 0:00:18.145433


In [4]:
%%time
bfcl_names = ['Header','View','Grain']+['pol'+str(i) for i in range(8)]+['tr_flag','n_pol','pred']
test_names = ['Header','View','Grain','Bar_sh','Phy']
for sb in ['sig','bckg']:
    for carb in ['c100','c60']:
        ya_bfcl = pd.read_csv(dir_path[carb]+carb+'_'+sb+'_pred_yandex.txt', names = bfcl_names)
        ya_sig = pd.read_csv(dir_path[carb]+carb+'_'+sb+'_grains.txt', names=test_names)
        res_bfcl = np.ones((0,ya_sig.shape[1]+1),dtype=int)
        i=0
        with open(dir_path[carb]+carb+'_'+sb+'_pred_grains.txt','w') as pfile:
            for _,row in ya_sig.iterrows():
                tmp_bfcl = ya_bfcl[ya_bfcl['View']==row['View']]
                #tmp_bfcl = tmp_bfcl[tmp_bfcl['View']==row['View']]
                if not (tmp_bfcl['Grain']==row['Grain']).any():
                    i+=1
                    if not i%100: print(i,'\n',row,'\n\n')
                tmp_bfcl = tmp_bfcl[tmp_bfcl['Grain']==row['Grain']]
                res_bfcl = np.vstack((res_bfcl,np.append(row.values,tmp_bfcl['pred'].values)))
                line = ','.join([str(k) if (k-int(k)) else str(int(k)) for k in np.append(row.values,tmp_bfcl['pred'].values)])
                pfile.write(line+'\n')
        #np.savetxt(dir_path[carb]+carb+'_'+sb+'_res_yandex.txt',res_bfcl,fmt='%d',delimiter=',')

CPU times: user 38.1 s, sys: 8.81 ms, total: 38.1 s
Wall time: 38.3 s
